In [1]:
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, GPTNeoModel
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pronouncing
from transformers import Trainer, TrainingArguments
from tqdm import tqdm
import random

import warnings
warnings.filterwarnings('ignore')

#Download Finetuned GPT-Neo
# Set the random seed to a fixed value to get reproducible results 
torch.manual_seed(42)
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", 
                                          bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")

# Download the pre-trained GPT-Neo model and transfer it to the GPU
model = GPTNeoForCausalLM.from_pretrained("FigoMe/news-gpt-neo-1.3B-keywords-line-by-line-reverse").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))

def get_stress(phone):
    stress = []
    for s in phone.split():
        if s[-1].isdigit():
            if s[-1] == '2':
                stress.append(0)
            else:
                stress.append(int(s[-1]))
    return stress

def alternating(stress):
    #Check if the stress and unstress are alternating
    check1 = len(set(stress[::2])) <= 1 and (len(set(stress[1::2])) <= 1)
    check2 = len(set(stress)) == 2 if len(stress) >=2 else True
    return (check1 and check2)

def get_phones(rhyme_word):
    phone = pronouncing.phones_for_word(rhyme_word)[0]
    stress = get_stress(phone)
    p_state = stress[0]
    n_syllables = len(stress)
    return p_state, n_syllables

from torch import Tensor
from torch.nn import functional as F


def top_k_top_p_filtering(
    logits: Tensor,
    top_k: int = 0,
    top_p: float = 1.0,
    filter_value: float = -float("Inf"),
    min_tokens_to_keep: int = 1,
    return_index = False
) -> Tensor:
    """Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
    Args:
        logits: logits distribution shape (batch size, vocabulary size)
        if top_k > 0: keep only top k tokens with highest probability (top-k filtering).
        if top_p < 1.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
            Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        Make sure we keep at least min_tokens_to_keep per batch example in the output
    From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    if top_k > 0:
        top_k = min(max(top_k, min_tokens_to_keep), logits.size(-1))  # Safety check
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        indices_keep = logits >= torch.topk(logits, top_k)[0][..., -1, None]
        indices_keep = indices_keep[0].tolist()
        indices_keep = [i for i,x in enumerate(indices_keep) if x == True]
        logits[indices_to_remove] = filter_value

    if top_p < 1.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold (token with 0 are kept)
        sorted_indices_to_remove = cumulative_probs > top_p
        if min_tokens_to_keep > 1:
            # Keep at least min_tokens_to_keep (set to min_tokens_to_keep-1 because we add the first one below)
            sorted_indices_to_remove[..., :min_tokens_to_keep] = 0
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # scatter sorted tensors to original indexing
        indices_to_remove = sorted_indices_to_remove.scatter(-1, sorted_indices, sorted_indices_to_remove)
        logits[indices_to_remove] = filter_value
    if return_index == True:
        return logits, indices_keep
    return logits


def reverse_order(line):
    line = line.replace(', ', ' , ')
    words = line.split()
    return ' '.join(reversed(words)).replace(' , ', ', ')


2023-12-14 01:11:39.425838: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-14 01:11:39.425940: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-14 01:11:39.571544: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-14 01:11:39.849017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
loose_list = ['that','is','of','the','it','a','as','with','like','go','to','on','in','at','are','and']
def check_either_stress(stress, source_word, loose = True):
    if loose and source_word in loose_list:
        return True
    if len(stress) == 1 and len(pronouncing.phones_for_word(source_word))>1:
                    phone0 = pronouncing.phones_for_word(source_word)[0]
                    phone1 = pronouncing.phones_for_word(source_word)[1]
                    stress0 = [int(s[-1]) for s in phone0.split() if s[-1].isdigit()]
                    stress1 = [int(s[-1]) for s in phone1.split() if s[-1].isdigit()]
                    if stress0+stress1 ==1 and stress0*stress1 == 0:
                        return True

    return False

In [4]:
def generate_next_word(input_ids1, temperature = 0.85, topk = 100, n_sample=10, device = 'cuda:0'):
    current_word = 0
    original = tokenizer.decode(input_ids1[0])
    for _ in range(1):
        outputs1 = model(input_ids1)
        #print(outputs1)
        next_token_logits1 = outputs1[0][:, -1, :]
        next_token_logits1 = top_k_top_p_filtering(next_token_logits1, top_k=topk)
        logit_zeros = torch.zeros(len(next_token_logits1)).cuda()
        #logit_zeros = torch.zeros(len(next_token_logits1), device=device)

        next_token_logits = next_token_logits1 * (1/ temperature)
        probs = F.softmax(next_token_logits, dim=-1)
        next_tokens = torch.multinomial(probs, num_samples=n_sample).squeeze(1)
        #unfinished_sents = torch.ones(1, dtype=torch.long, device=device)
        unfinished_sents = torch.ones(1, dtype=torch.long).cuda()
        tokens_to_add = next_tokens * unfinished_sents + tokenizer.pad_token_id * (1 - unfinished_sents)

        temp = []
        for i in range(len(input_ids1)):
            temp +=[torch.cat([input_ids1[i].reshape(1,-1), token_to_add.reshape(1,-1)], dim=-1) for token_to_add in tokens_to_add[i]]
        input_ids1 = torch.stack(temp).view(len(temp),-1)
        # decode the generated token ids to natural words
        results = []
        input_ids1_l = []
        for input_id1 in input_ids1:
            gen = tokenizer.decode(input_id1).replace(original,'').strip(' ')
            if len(gen.split()) >0:
                gen = gen.split()[0]
                gen = gen.lower()
                if gen not in results:
                    results.append(gen)
        return results
        '''
        if tokenizer.decode(tokens_to_add[0])[0] == ' ':
            if current_word ==1:
                return tokenizer.decode(input_ids1[0]).split()[-1], False
            current_word += 1
        input_ids1 = torch.cat([input_ids1, tokens_to_add.unsqueeze(-1)], dim=-1)
        '''

In [5]:
device = 'cuda:0'
from transformers import AutoTokenizer, AutoModelForCausalLM
gpt2_tokenizer  = AutoTokenizer.from_pretrained('gpt2-large')
gpt2_model = AutoModelForCausalLM.from_pretrained('gpt2-large')
gpt2_model = gpt2_model.to(device)
gpt2_model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1280)
    (wpe): Embedding(1024, 1280)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-35): 36 x GPT2Block(
        (ln_1): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1280, out_features=50257, bias=False)
)

Update 1201: 
- dealing with a phrase
- improvements of keywords enforcement
- sample from the final beam

made changes to the below functions:

In [6]:
def regularBeamSearch(prompts):
	'''
	Beam search that considers the coherence by adding a new variable: previously_generated_lines
	'''
	BeamScorer = {}
	for sentence in prompts:
		loss = score_gpt2(sentence)
		BeamScorer[sentence] = [loss]
	answers = sorted(BeamScorer.items(), key=lambda x: x[1], reverse=False)
	new_prompts = [ans[0] for ans in answers]
	return new_prompts

In [7]:
softmax = torch.nn.Softmax(dim=1)
def sample_prompts(prompts,previous='', temperature = 1):
    BeamScorer = {}
    for sentence in prompts:
        loss = score_gpt2(previous+sentence)
        BeamScorer[sentence] = [loss]
    p = BeamScorer.values()
    p = torch.tensor(list(p))*(1/temperature)
    try:
        p = p.squeeze(1)
    except:
        pass
    p_softmax = torch.nn.functional.softmax(p)
    index = torch.multinomial(p_softmax,num_samples=len(prompts))
    new_prompts = [prompts[i] for i in index]
    return new_prompts

In [8]:
def score_gpt2(sentence, normalize = True):
	'''
	The default setting is to normalize because we won't face the issue mentioned in function "score".
	'''
	tokens_tensor = gpt2_tokenizer.encode(sentence, add_special_tokens=False, return_tensors="pt")[0].cuda()
	with torch.no_grad():
		loss = gpt2_model(tokens_tensor, labels=tokens_tensor)[0]
	if normalize:
		return loss/len(tokens_tensor)
	else:
		return loss

In [9]:
def myBeamSearch(prompts, all_states, all_n_sys, all_keywords, beam_size = 5,enforce_keywords=True):
    BeamScorer = {}
    return_seq, return_stt, return_sys, return_key = [], [], [], []
    
    if (not enforce_keywords) or len(all_keywords)==0:
        for sentence, p_state, n_sys, keywords in zip(prompts, all_states, all_n_sys, all_keywords):
            loss = score(sentence)
            BeamScorer[sentence] = [loss, p_state, n_sys, keywords]
        answers = sorted(BeamScorer.items(), key=lambda x: x[1], reverse=False)
    else:
        min_remaining = min([len(x) for x in all_keywords])
        for sentence, p_state, n_sys, keywords in zip(prompts, all_states, all_n_sys, all_keywords):
            #start with fewer keywords remaining
            if len(keywords) == min_remaining:
                loss = score(sentence)
                BeamScorer[sentence] = [loss, p_state, n_sys, keywords]
        answers = sorted(BeamScorer.items(), key=lambda x: x[1], reverse=False)
        BeamScorer={}
        for sentence, p_state, n_sys, keywords in zip(prompts, all_states, all_n_sys, all_keywords):
            #then
            if len(keywords) == min_remaining+1:
                loss = score(sentence)
                BeamScorer[sentence] = [loss, p_state, n_sys, keywords]
        answers += sorted(BeamScorer.items(), key=lambda x: x[1], reverse=False)
        BeamScorer={}
        for sentence, p_state, n_sys, keywords in zip(prompts, all_states, all_n_sys, all_keywords):
            #last, most keywords remaining
            if len(keywords) == min_remaining+2:
                loss = score(sentence)
                BeamScorer[sentence] = [loss, p_state, n_sys, keywords]
        answers += sorted(BeamScorer.items(), key=lambda x: x[1], reverse=False)
    new_prompts = [ans[0] for ans in answers]
    new_p_states = [ans[1][1] for ans in answers]
    new_n_sys = [ans[1][2] for ans in answers]
    new_keywords = [ans[1][3] for ans in answers]
    l = len(new_prompts)
    if l > beam_size:
        return_seq += new_prompts[0:beam_size]
        return_stt += new_p_states[0:beam_size]
        return_sys += new_n_sys[0:beam_size]
        return_key += new_keywords[0:beam_size]
    else:
        return_seq +=new_prompts
        return_stt += new_p_states
        return_sys += new_n_sys
        return_key += new_keywords
    return return_seq,return_stt, return_sys, return_key


In [10]:
def score(sentence, normalize = True):
	'''
	Score a single sentence using the plan-to-lyrics model.
	The recommended setting is to NOT normalize, because the input sentence is very long: it contains the title, planed keywords, and previously generated lines. 
	In addition, the candidate sentences contain the same prefix (i.e., the title, planed keywords, and previously generated lines) and only differ in the currently generated line.
	Normaling means dividing the loss by a large factor which may result in similarity accross different candidate sentences.
	'''
	tokens_tensor = tokenizer.encode(sentence, add_special_tokens=False, return_tensors="pt")[0].cuda()
	with torch.no_grad():
		loss = model(tokens_tensor, labels=tokens_tensor)[0]
	if normalize:
		return loss/len(tokens_tensor)
	else:
		return loss

In [11]:
def get_stress_phrase(phrase):
    words = phrase.split()
    stress=[]
    for source_word in words:
        phone = pronouncing.phones_for_word(source_word)[0]
        stress+= get_stress(phone)
    return stress

In [12]:
single_character_word = ['i','a']
forbidden_words = ['dona','er','ira','ia',"'s","'m","hmm","mm"]
def get_valid_samples(prompt, p_state, n_syllables, keywords, n_sample=30, n_cands=5):
    #if n_syllables == 10 or n_syllables==11:
    if n_syllables == 10 and len(keywords)==0:
        return [prompt], [p_state], [n_syllables], [keywords]
    elif n_syllables > 10:
        return [], [], [],[]
    states = []
    all_n_syl = []
    
    prompts = []
    all_keywords= [] 
    #insert the keyword whenever possible
    for source_word in keywords:
        stress = get_stress_phrase(source_word)
        #if not alternating(stress):
            #continue

        #if the word is single syllable and can be either stressed or unstressed, flag = True
        flag = check_either_stress(stress, source_word)

        if (stress[-1] == 1- p_state or flag) and (n_syllables+len(stress)<=10):
            states.append(stress[0])
            all_n_syl.append(n_syllables+len(stress))
            #print(source_word)
            prompts.append(prompt+ ' ' +reverse_order(source_word))
            copy = keywords.copy()
            copy.remove(source_word)
            all_keywords.append(copy)    
    
    #The normal process of decoding
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
    tokens = generate_next_word(input_ids, n_sample=n_sample)
    #print(tokens)
    for token in tokens:
        token = token.lower()
        if (len(token) == 1 and token not in single_character_word) or token in forbidden_words:
            continue
        if token not in prompt:
            try:
                phone = pronouncing.phones_for_word(token)[0]
                stress = get_stress(phone)
            except:
                continue
            if (not alternating(stress)) or (len(stress)==0):
                continue

            #if the word is single syllable and can be either stressed or unstressed, flag = True
            flag = check_either_stress(stress, token)
            if n_syllables+len(stress)<=10:
                if (stress[-1] == 1- p_state) or flag:
                    tokens.append(token)
                    if stress[-1] == 1- p_state:
                        states.append(stress[0])
                    elif flag:
                        states.append(1- p_state)
                    all_n_syl.append(n_syllables+len(stress))
                    prompts.append(prompt+ ' ' + token )
                    all_keywords.append(keywords)
                    if len(prompts)>= n_cands:
                        return prompts, states, all_n_syl, all_keywords
    return prompts, states, all_n_syl, all_keywords

In [15]:
four_seasons_story_line = [
['susie', 'nature', 'loved'],
['she', 'to', 'wanted'],
['beach', 'her', 'beloved'],
['sunset', 'beautiful', 'undaunted'], 
['relaxing', 'day', 'spent'],
['nervous', 'were', 'both'],
['couple', 'talked', 'sent'],
['back', 'home', 'growth'],
['quiet', 'afterwards', 'felt'],
['alone like a stream', 'glad', 'now'],
['sleep', 'could', 'belt'],
['asleep', 'night', 'how'],
['sleeping', 'restless', 'feeling'],
['wake', 'morning', 'appealing']]


example_title = 'solitary contemplations in nature'
beam_size=20
previous = ""
enforce_keywords = True
for kws in tqdm(four_seasons_story_line):
    success=False
    n_sample = 30
    while success != True:
        print(kws)
        rhyme_word = kws[-1]
        prefix =  '''Keywords: ''' + '; '.join(kws) +'. Sentence in reverse order: '
        prompt = '''<|startoftext|> Title: ''' + example_title + ' ' + ','.join(previous.split(',')[-3:]) + prefix + rhyme_word
        #prompt = '''<|startoftext|> Title: ''' + example_title + ' ' + prefix + rhyme_word
        p_state, n_syllables = get_phones(rhyme_word)
        result_list = []
        i=0
        prompts, all_states, all_n_sys, all_keywords = get_valid_samples(prompt,p_state, n_syllables, keywords = kws[:2], n_sample=n_sample,n_cands=5)
        while i<7:
            #print(i)
            new_prompts, new_states, new_n_sys, new_keywords = [], [], [], []
            for prompt, p_state, n_syllables, keyword in zip(prompts, all_states, all_n_sys, all_keywords):
                t_p, t_state, t_sys, t_keywords = get_valid_samples(prompt, p_state, n_syllables, keyword,n_sample=n_sample)
                new_prompts+=t_p
                new_states+=t_state
                new_n_sys+=t_sys
                new_keywords+=t_keywords
            prompts, all_states, all_n_sys, all_keywords = new_prompts, new_states, new_n_sys, new_keywords
            prompts, all_states, all_n_sys, all_keywords = myBeamSearch(prompts,all_states, all_n_sys, all_keywords, beam_size=beam_size, enforce_keywords=enforce_keywords)
            i += 1
        if len(prompts)==0:
            if n_sample>300:
                print('Failed to generate valid samples. Please try re-generation for this line.')
                previous += '   ,'
                break
            n_sample = n_sample*3

        else:
            correct_prompts = [reverse_order(p.split('order: ')[1]) for p in prompts]
            print(correct_prompts)
            result_list = sample_prompts(correct_prompts, previous)
            
            success=True
            found = False 
            for r in result_list:
                if kws[0] in r or kws[1] in r:
                    previous = previous + r + ','
                    found = True
                    break
            if found == False:
                    previous = previous + result_list[0]+','
                    n_sample = n_sample*3


  0%|                                                    | 0/14 [00:00<?, ?it/s]

['susie', 'nature', 'loved']
['that was the nature susie known and loved', 'with plants and all the nature susie loved', 'that is the nature susie known and loved', 'as plants and all the nature susie loved', 'as is the nature susie known and loved', 'and how the cruel nature susie loved', 'the wild and cruel nature susie loved', 'the harsh and cruel nature susie loved', 'that world and all the nature susie loved', 'is how the cruel nature susie loved', 'and beauty that the nature susie loved', 'that was the very nature susie loved', 'like how the cruel nature susie loved', 'that is the very nature susie loved', 'and all that cruel nature susie loved', 'with all that cruel nature susie loved', 'that harsh and cruel nature susie loved', 'and was the very nature susie loved', 'and silence that the nature susie loved']


  7%|███▏                                        | 1/14 [00:27<06:02, 27.89s/it]

['she', 'to', 'wanted']
['let go to as she is always wanted', 'would go to as she is always wanted', 'not go to as she is always wanted', 'her go to as she is always wanted', 'much as to that she is always wanted', 'all go to as she is always wanted', 'let go to that she is always wanted', 'with care as to she is always wanted', 'care as to that she is always wanted', 'close as to that she is always wanted', 'with her as to she is always wanted', 'did go to that she is always wanted', 'life as to that she is always wanted', 'her go to that she is always wanted', 'you go to that she is always wanted', 'will go to that she is always wanted', 'man as to that she is always wanted', 'that world as to she is always wanted', 'able to as she is always wanted', 'going to as she is always wanted']


 14%|██████▎                                     | 2/14 [01:06<06:53, 34.44s/it]

...

 86%|████████████████████████████████████▊      | 12/14 [07:01<01:09, 34.81s/it]

['go into night asleep remember how', 'asleep another night imagine how']
['sleeping', 'restless', 'feeling']
['likely was the restless sleeping feeling', 'you ignore the restless sleeping feeling', 'well except the restless sleeping feeling', 'clearly was the restless sleeping feeling', 'reason for the restless sleeping feeling', 'we ignore the restless sleeping feeling', 'was the always restless sleeping feeling', 'really was the restless sleeping feeling', 'know except the restless sleeping feeling', 'just ignore the restless sleeping feeling', 'we are always restless sleeping feeling', 'second was the restless sleeping feeling', 'us are always restless sleeping feeling', 'from the restless sleeping troubled feeling', 'culprit for the restless sleeping feeling', 'you are always restless sleeping feeling', 'was the restless sleeping troubled feeling', 'you are really restless sleeping feeling', 'kids are always restless sleeping feeling', 'things except the restless sleeping feeling'

 93%|███████████████████████████████████████▉   | 13/14 [07:33<00:34, 34.13s/it]

['wake', 'morning', 'appealing']
['but that morning wake is not appealing', 'why that morning wake is not appealing', 'and that morning wake is not appealing', 'now that morning wake is not appealing', 'well that morning wake is not appealing', 'there and morning wake is not appealing', 'noon and morning wake is not appealing', 'want and morning wake is not appealing', 'you and morning wake is not appealing', 'its and morning wake is not appealing', 'always wake that morning not appealing', 'never wake that morning not appealing', 'always wake that morning quite appealing', 'even wake that morning is appealing', 'never wake that morning quite appealing', 'even morning wake is not appealing', 'little morning wake is not appealing', 'even wake that morning not appealing', 'truly wake that morning is appealing', 'other wake that morning is appealing']


100%|███████████████████████████████████████████| 14/14 [08:07<00:00, 34.80s/it]


In [16]:
print('Enforce keywords:\n')

print(previous.replace(',',',\n'))

Enforce keywords:

the wild and cruel nature susie loved,
not go to as she is always wanted,
towards that lonely beach with her beloved,
and beautiful sunset went undaunted,
the joy as your relaxing day is spent,
that people who are nervous were with both,
another couple talked and also sent,
go back is home observing our growth,
we are just as quiet afterwards felt,
glad alone like a stream the silence now,
is sit and sleep that could with our belt,
go into night asleep remember how,
just ignore the restless sleeping feeling,
other wake that morning is appealing,

